# GLOBAL

In [ ]:
import mdtools
import itertools
from simtk.unit import *
from simtk.openmm.app import ForceField, Topology

import mdtraj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# matplotlib.use('Agg')

from scipy import stats

import re
import sys
import glob
import ast

from new_functions import *

sns.set_theme(style="darkgrid")


%matplotlib inline

In [ ]:
import matplotlib as mpl
font_paths = mpl.font_manager.findSystemFonts()
font_objects = mpl.font_manager.createFontList(font_paths)
font_names = [f.name for f in font_objects]
print (font_names)

sns.set(font="Cantarell")

In [ ]:
dpi = 300

# IONS

In [ ]:
# Check which simulations have been run

for cat in ['291.15', '293.15', '60A_293.15', '150A_293.15']:
    print(cat)
    sigma = pd.read_csv(glob.glob(f'../md_out/ions/{cat}/*.txt')[0], names=['path', 'sigma', 'a', 'b', 'c'])['path'].tolist()
    sigma = [i.split('/')[-1] for i in sigma]
    sigma = [i.split('-') for i in sigma]
    sigma = pd.DataFrame(sigma).drop(columns=[2])
    
    h5 = [i.split('/')[-1] for i in glob.glob(f'../md_out/ions/{cat}/*.h5')]
    h5 = [i.split('-') for i in h5]
    h5 = pd.DataFrame(h5).drop(columns=[2])
    print('Missing')
    print(0.172)
    a_lesser = h5[h5[0] == '0.172M'][1].sort_values(ascending=True).unique().tolist()
    b_lesser = sigma[sigma[0] == '0.172M'][1].sort_values(ascending=True).unique().tolist()
    print(np.setdiff1d(a_lesser,b_lesser))
    
    print(0.346)
    a_greater = h5[h5[0] == '0.346M'][1].sort_values(ascending=True).unique().tolist()
    b_greater = sigma[sigma[0] == '0.346M'][1].sort_values(ascending=True).unique().tolist()
    print(np.setdiff1d(a_greater,b_greater))
    print('h5s')
    print(0.172)
    print(a_lesser)

    print(0.346)
    
    print(a_greater)
    print()

# IONS KDE

In [ ]:
fig, axes= plt.subplots(3,1, figsize=(8.5,11.5))

for i in ['0', '5.0', '10.0', '15.0', '20.0', '25.0']:
    df = pd.read_csv(f'../md_out/ions/293.15/0.172M-{i}-b.csv')
    sns.kdeplot(data=df, x='Temperature (K)', ax=axes[0], label=f'{i} MV/cm')
    sns.lineplot(data=df, x='Time (ps)', y='Total Energy (kJ/mole)', ax=axes[1], label=f'{i} MV/cm')
    sns.lineplot(data=df, x='Time (ps)', y='Temperature (K)', ax=axes[2], label=f'{i} MV/cm')

axes[0].legend()
axes[1].legend()
axes[2].legend()

plt.suptitle('Thermostat breaks down at high EF 0.172')
plt.show()

In [ ]:
df = pd.DataFrame()

# for i in ['0.08']:
for i in ['0', '5.0', '10.0', '15.0', '20.0', '25.0']:
    temp = pd.read_csv(f'../md_out/ions/temp/0.346M-{i}-a.csv')
    temp['MV/cm'] = f'{i} MV/cm'
    df = pd.concat([df, temp])

fig, axes= plt.subplots(3,1, figsize=(8.5,11.5), dpi=dpi)

sns.kdeplot(data=df, x='Temperature (K)', ax=axes[0],
            hue='MV/cm', palette='mako')
sns.lineplot(data=df, x='Time (ps)', y='Total Energy (kJ/mole)', ax=axes[2], 
             hue='MV/cm', palette='mako')
sns.lineplot(data=df, x='Time (ps)', y='Temperature (K)', ax=axes[1], 
             hue='MV/cm', palette='mako')

# axes[0].legend()
axes[1].get_legend().remove()
axes[2].get_legend().remove()

plt.suptitle('Thermostat Breaks Down at High EF (30A, 293.15K)', y=0.93)
# plt.tight_layout()
plt.savefig('!breakdown-30A.png')
plt.show()
plt.close(fig)

In [ ]:
atoms = mdtraj.load_frame('../md_out/ions/60A_293.15/0.346M-20.0-a.h5', 100)
ions = atoms.top.select('!protein and !water')
traj = mdtraj.load('../md_out/ions/60A_293.15/0.346M-25.0-a.h5', atom_indices=ions, stride=3)
diff = np.diff(traj.xyz, axis=0)
ok = diff.reshape((len(diff[:,0])*len(traj.xyz[0,:,0]),3))

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(8.5,4))

sns.histplot(ok[:,0], ax=axes[0])
sns.histplot(ok[:,1], ax=axes[1])


In [ ]:
'''
COMBINED
'''

fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

for color, cat, label in zip(['#3CAEA3', 'dodgerblue', 'steelblue', '#34568B'], 
                             ['291.15', '293.15', '60A_293.15', '150A_293.15'], 
                             ['30A @ 291.15', '30A @ 293.15', '60A @ 293.15', '150A @ 293.15']):

    # Load in Data
    df = pd.read_csv(f'../md_out/ions/{cat}/x_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
    # MV/cm
    df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
    # Molarity
    df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
    # mS/cm
    df['sigma'] = [i*-1000 for i in df['sigma'].tolist()]


    lesser = df[df['a'] == '0.172M'].copy()
    greater = df[df['a'] == '0.346M'].copy()

    # Lesser
    sns.lineplot(data=lesser, x='c', y='sigma', ax=axes[0], color=color, err_style='band')
    

    for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
        sns.scatterplot(data=lesser[(lesser['c'] >= i) & (lesser['c']< j)], x='c', y='sigma', ax=ax, color=color, label=f'{label}')

for ax in axes.reshape(-1):    
    ax.axhline(16, linestyle='--', color='salmon', label='Literature Value')
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set_yscale('log')  
    
axes[0].legend().set_visible(False)
axes[1].legend().set_visible(False)
axes[2].legend().set_visible(False)
axes[3].legend(loc='upper left', fontsize=8, markerscale=.66)
axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (0.172M)', fontsize=15, y=0.92)
plt.savefig('!combined_lesser.png')
plt.close(fig)

fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

for color, cat, label in zip(['#3CAEA3', 'dodgerblue', 'steelblue', '#34568B'], 
                             ['291.15', '293.15', '60A_293.15', '150A_293.15'], 
                             ['30A @ 291.15', '30A @ 293.15', '60A @ 293.15', '150A @ 293.15']):

    # Load in Data
    df = pd.read_csv(f'../md_out/ions/{cat}/x_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
    # MV/cm
    df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
    # Molarity
    df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
    # mS/cm
    df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]


    lesser = df[df['a'] == '0.172M'].copy()
    greater = df[df['a'] == '0.346M'].copy()

    # Lesser
    sns.lineplot(data=greater, x='c', y='sigma', ax=axes[0], color=color, err_style='band')
    

    for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
        sns.scatterplot(data=greater[(greater['c'] >= i) & (greater['c']< j)], x='c', y='sigma', ax=ax, color=color, label=f'{label}')

for ax in axes.reshape(-1):    
    ax.axhline(30.2, linestyle='--', color='salmon', label='Literature Value')
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set_yscale('log')    
    
axes[0].legend().set_visible(False)
axes[1].legend().set_visible(False)
axes[2].legend().set_visible(False)
axes[3].legend(loc='upper left', fontsize=8, markerscale=.66)
axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (0.346M)', fontsize=15, y=0.92)
plt.savefig('!combined_greater.png')
plt.close(fig)

In [ ]:
'''
NACL 293.15
'''

# Load in Data
df = pd.read_csv('../md_out/ions/293.15/x_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
# MV/cm
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
# Molarity
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
# mS/cm
df['sigma'] = [i*1000 for i in df['sigma'].tolist()]


lesser = df[df['a'] == '0.172M'].copy()
greater = df[df['a'] == '0.346M'].copy()

# Lesser
fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(data=lesser, x='c', y='sigma', ax=axes[0], color='dodgerblue', err_style='band')
axes[0].axhline(-16, linestyle='--', color='salmon', label='Literature Value')
axes[0].set(ylabel=r'Conductivity (mS/cm)')
axes[0].legend()

for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
#     ax2 = ax.twinx()
#     ax2.set(ylabel=f'{i} to {j} MV/cm')
    
#     ax2.tick_params(
#     axis='y',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     labelrotation=180,      # ticks along the bottom edge are off
#     right=False,         # ticks along the top edge are off
#     labelright=False
#     grid_alpha=0) # labels along the bottom edge are off
    
    sns.scatterplot(data=lesser[(lesser['c'] >= i) & (lesser['c']< j)], x='c', y='sigma', ax=ax, color='dodgerblue')
    ax.axhline(-16, linestyle='--', color='salmon', label='Literature Value')
    ax.legend()
    ax.set(ylabel=r'Conductivity (mS/cm)')
    
axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (l=30A, M=0.172, T=293.15)', fontsize=15, y=0.92)
plt.savefig('!NaCl-293.15-0.172M.png')
plt.close(fig)

# Greater

fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(data=greater, x='c', y='sigma', ax=axes[0], color='dodgerblue', err_style='band')
axes[0].axhline(-16, linestyle='--', color='salmon', label='Literature Value')
axes[0].set(ylabel=r'Conductivity (mS/cm)')
axes[0].legend()

for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
    sns.scatterplot(data=greater[(greater['c'] >= i) & (greater['c']< j)], x='c', y='sigma', ax=ax, color='dodgerblue')
    ax.axhline(-30.2, linestyle='--', color='salmon', label='Literature Value')
    ax.legend()
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (l=30A, M=0.346, T=293.15)', fontsize=15, y=0.92)
plt.savefig('!NaCl-293.15-0.346M.png')
plt.close(fig)

In [ ]:
'''
NACL 60A_293.15
'''

# Load in Data
df = pd.read_csv('../md_out/ions/60A_293.15/x_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
# MV/cm
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
# Molarity
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
# mS/cm
df['sigma'] = [i*1000 for i in df['sigma'].tolist()]


lesser = df[df['a'] == '0.172M'].copy()
greater = df[df['a'] == '0.346M'].copy()

# Lesser
fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(data=lesser, x='c', y='sigma', ax=axes[0], color='dodgerblue', err_style='band')
axes[0].axhline(-16, linestyle='--', color='salmon', label='Literature Value')
axes[0].set(ylabel=r'Conductivity (mS/cm)')
axes[0].legend()

for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
    sns.scatterplot(data=lesser[(lesser['c'] >= i) & (lesser['c']< j)], x='c', y='sigma', ax=ax, color='dodgerblue')
    ax.axhline(-16, linestyle='--', color='salmon', label='Literature Value')
    ax.legend()
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (l=60A, M=0.172, T=293.15)', fontsize=15, y=0.92)
plt.savefig('!NaCl-60A-293.15-0.172M.png')
plt.close(fig)

# Greater

fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(data=greater, x='c', y='sigma', ax=axes[0], color='dodgerblue', err_style='band')
axes[0].axhline(-16, linestyle='--', color='salmon', label='Literature Value')
axes[0].set(ylabel=r'Conductivity (mS/cm)')
axes[0].legend()

for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
    sns.scatterplot(data=greater[(greater['c'] >= i) & (greater['c']< j)], x='c', y='sigma', ax=ax, color='dodgerblue')
    ax.axhline(-30.2, linestyle='--', color='salmon', label='Literature Value')
    ax.legend()
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (l=60A, M=0.346, T=293.15)', fontsize=15, y=0.92)
plt.savefig('!NaCl-60A-293.15-0.346M.png')
plt.close(fig)

In [ ]:
'''
NACL 291.15
'''

# Load in Data
df = pd.read_csv('../md_out/ions/291.15/x_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
# MV/cm
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
# Molarity
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
# mS/cm
df['sigma'] = [i*1000 for i in df['sigma'].tolist()]


lesser = df[df['a'] == '0.172M'].copy()
greater = df[df['a'] == '0.346M'].copy()

# Lesser
fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(data=lesser, x='c', y='sigma', ax=axes[0], color='dodgerblue', err_style='band')
axes[0].axhline(-16, linestyle='--', color='salmon', label='Literature Value')
axes[0].set(ylabel=r'Conductivity (mS/cm)')
axes[0].legend()

for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
    sns.scatterplot(data=lesser[(lesser['c'] >= i) & (lesser['c']< j)], x='c', y='sigma', ax=ax, color='dodgerblue')
    ax.axhline(-16, linestyle='--', color='salmon', label='Literature Value')
    ax.legend()
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (l=30A, M=0.172, T=293.15)', fontsize=15, y=0.92)
plt.savefig('!NaCl-291.15-0.172M.png')
plt.close(fig)

# Greater

fig, axes = plt.subplots(4,1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(data=greater, x='c', y='sigma', ax=axes[0], color='dodgerblue', err_style='band')
axes[0].axhline(-16, linestyle='--', color='salmon', label='Literature Value')
axes[0].set(ylabel=r'Conductivity (mS/cm)')
axes[0].legend()

for ax, i, j in zip(axes.reshape(-1)[1:],[0,0.1,1.5],[0.1,1.5,np.inf]):
    sns.scatterplot(data=greater[(greater['c'] >= i) & (greater['c']< j)], x='c', y='sigma', ax=ax, color='dodgerblue')
    ax.axhline(-30.2, linestyle='--', color='salmon', label='Literature Value')
    ax.legend()
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[3].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of NaCl Solutions (l=30A, M=0.346, T=293.15)', fontsize=15, y=0.92)
plt.savefig('!NaCl-291.15-0.346M.png')
plt.close(fig)

# HEWL

In [ ]:
glob.glob('../HEWLmd/run1/*.csv')

In [ ]:
fig, axes= plt.subplots(3,1, figsize=(8.5,11.5))

for i in ['0', '5.0', '10.0', '15.0', '20.0', '25.0']:
    df = pd.read_csv(f'../HEWLmd/run2/z_conductivity_results.txt')
    sns.kdeplot(data=df, x='Temperature (K)', ax=axes[0], label=f'{i} MV/cm')
    sns.lineplot(data=df, x='Time (ps)', y='Total Energy (kJ/mole)', ax=axes[1], label=f'{i} MV/cm')
    sns.lineplot(data=df, x='Time (ps)', y='Temperature (K)', ax=axes[2], label=f'{i} MV/cm')

In [ ]:
hewl_df = pd.DataFrame()

for i in range(1,4):
    df = pd.read_csv(f'../HEWLmd/run{i}/z_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
    df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
    df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
    df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]
    hewl_df = pd.concat([hewl_df, df])

fig, axes = plt.subplots(3,1, figsize=(8.5,11.5))

sns.lineplot(data=hewl_df, x='c', y='sigma', ax=axes[0], color='dodgerblue', label='Experimental Values')
sns.scatterplot(data=hewl_df, x='c', y='sigma', ax=axes[1], color='dodgerblue', label='Experimental Values')
sns.scatterplot(data=hewl_df[hewl_df['c'] > 0.09], x='c', y='sigma', ax=axes[2], color='dodgerblue', label='Experimental Values')

for ax in axes.reshape(-1):
    ax.axhline(0.053, linestyle='--', color='salmon', label='Morosova Value')
    ax.set(ylabel=r'Conductivity (mS/cm)')

axes[0].legend(loc='lower right')
axes[1].legend().set_visible(False)
axes[2].legend().set_visible(False)


axes[2].set(xlabel=r'Electric-field (MV/cm)')
plt.suptitle('Conductivity of HEWL Crystal', fontsize=15, y=0.92)

plt.savefig('temp.png')
plt.show()
plt.close(fig)

In [ ]:
# HEWLmd

fig, axes = plt.subplots(3,1, figsize=(8.5,11.5))


df = pd.read_csv('../HEWLmd/run2/z_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]

sns.lineplot(data=df, x='c', y='sigma', ax=axes[0], color='dodgerblue', label='Conductivity')
axes[0].axhline(0.053, linestyle='--', color='salmon', label='Literature Value')

sns.scatterplot(data=df[df['c'] > 0.09], x='c', y='sigma', ax=axes[1], color='dodgerblue', label='Conductivity')


df = pd.read_csv('../HEWLmd/run1/z_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]
sns.lineplot(data=df, x='c', y='sigma', ax=axes[0], color='#3CAEA3', label='Conductivity')

sns.scatterplot(data=df[df['c'] > 0.09], x='c', y='sigma', ax=axes[1], color='#3CAEA3', label='Conductivity')

df = pd.read_csv('../HEWLmd/run3/z_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]
sns.lineplot(data=df, x='c', y='sigma', ax=axes[0], color='steelblue', label='Conductivity')
sns.scatterplot(data=df[df['c'] > 0.09], x='c', y='sigma', ax=axes[1], color='steelblue', label='Conductivity')
axes[1].axhline(0.053, linestyle='--', color='salmon', label='Literature Value')


plt.savefig('temp.png')
plt.show()
plt.close(fig)

# ALIGNMENT

In [ ]:
file = glob.glob(f'../md_out/water/30A/*.h5')[0]
itertraj = mdtraj.iterload(f'{file}', 300)
traj = next(itertools.islice(itertraj, 30))
t = traj.atom_slice(traj.top.select("water"))

In [ ]:
# RHO MAGNITUDE

huh = []
yuy = []

for i in (data[:,-1]):
    file = glob.glob(f'../md_out/water/0M-{i}-c.h5')[0]
    itertraj = mdtraj.iterload(f'{file}', 300)
    traj = next(itertools.islice(itertraj, 30))
    t = traj.atom_slice(traj.top.select("water"))

    h_pos = (t.xyz[:,1::3,:] + t.xyz[:,2::3,:]) / 2
    dipole_direction = t.xyz[:,0::3] - h_pos
    frame_average_dipole = np.mean(dipole_direction, axis=0)

    rho = np.mean(np.sqrt(np.sum((frame_average_dipole ** 2), axis=1)))
    huh.append(rho)
    yuy.append(i)

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(10,10))
sns.lineplot(data=align_df_x, x='ef', y='theta', ax=axes[0], color='dodgerblue')
# sns.scatterplot(data=align_df_x, x='ef', y='theta', ax=axes[0])

sns.scatterplot(data=align_df_x[align_df_x['ef'] < 5], x='ef', y='theta', ax=axes[0], alpha=0.3, color='gray')
# sns.lineplot(data=align_df[align_df['ef'] < 5], x='ef', y='theta', ax=axes)
sns.lineplot(x=np.linspace(0.01,50,100), y=ok, ax=axes[0], color='salmon')


sns.lineplot(data=align_df_z, x='ef', y='theta', ax=axes[1], color='dodgerblue')
# sns.scatterplot(data=align_df_z, x='ef', y='theta', ax=axes[1])

sns.scatterplot(data=align_df_z[align_df_z['ef'] < 5], x='ef', y='theta', ax=axes[1], alpha=0.3, color='gray')
# sns.lineplot(data=align_df[align_df['ef'] < 5], x='ef', y='theta', ax=axes)
sns.lineplot(x=np.linspace(0.01,50,100), y=ok, ax=axes[1], color='salmon')

plt.show()

In [ ]:
align_df_x = pd.DataFrame()
align_df_z = pd.DataFrame()

for file in glob.glob(f'../md_out/water/*.h5'):
    
    df = pd.DataFrame()
    temp_theta_x = []
    temp_theta_z = []
        
    itertraj = mdtraj.iterload(f'{file}', 300)
    traj = next(itertools.islice(itertraj, 30))
    t = traj.atom_slice(traj.top.select("water"))

    h_pos = (t.xyz[:,1::3,:] + t.xyz[:,2::3,:]) / 2
    dipole_direction = t.xyz[:,0::3] - h_pos
    frame_average_dipole = np.mean(dipole_direction, axis=1)

    for i in frame_average_dipole:
        temp_theta_x.append(np.dot((i / np.linalg.norm(i)), [1,0,0]))
        temp_theta_z.append(np.dot((i / np.linalg.norm(i)), [0,0,1]))
        
    ef = np.full(len(temp_theta_x), float(file.split('0M')[-1].split('-')[1]))
    df['ef'] = ef
    
    df['theta'] = temp_theta_x
    align_df_x = pd.concat([align_df_x, df])
    df['theta'] = temp_theta_z
    align_df_z = pd.concat([align_df_z, df])    

In [ ]:
# ALIGNMENT

file = glob.glob(f'../md_out/water/30A/*.h5')[0]
itertraj = mdtraj.iterload(f'{file}', 300)
traj = next(itertools.islice(itertraj, 30))
t = traj.atom_slice(traj.top.select("water"))

h_pos = (t.xyz[:,1::3,:] + t.xyz[:,2::3,:]) / 2
dipole_direction = t.xyz[:,0::3] - h_pos
frame_average_dipole = np.mean(dipole_direction, axis=0)

rho = np.mean(np.sqrt(np.sum((frame_average_dipole ** 2), axis=1)))

ok = []

f = open('../md_out/water/30A/alignment_results.txt', "r")
for x in f:
    temp = x.replace('[', '')
    temp = temp.replace(']', '')
    temp = temp.replace("'", '')
    temp = temp.replace("\n", '')
    if temp[0] == ' ':
        temp = temp[1:]
    temp = temp.split(' ')
    
    
    for i in temp:
        try:
            ok.append(float(i))
        except:
            ok.append(i)

data = np.empty((86, 901))

for i in range(86):
    start = i*902
    end = (i+1)*902
    temp = ok[start:end]
    del temp[900]
    
    data[i] = temp

    
    
    
    
    
    
means = np.zeros(86)


ef = np.empty(0)
y = np.empty(0)
for count, i in enumerate(data):
    ef = np.append(ef, (len(i)-1)*[data[count,-1]])
    y = np.append(y, i[:-1])
    
for count, i in enumerate(data):
    means[count] = np.mean(i[:-1])
    
    
y /= rho
means /= rho    
    
    
def P_theta(theta, alpha):
    return (alpha/2)*np.exp(alpha*np.cos(theta))*np.sin(theta)/np.sinh(alpha)    
    
ok = []

for E in np.linspace(0.01,50,100):
    mu=6.2e-30  # Cm, see http://hyperphysics.phy-astr.gsu.edu/hbase/electric/diph2o.html
    E*=1e8       # J/Cm (or V/m); 1e8 corresponds to 1 MV/cm
    k=1.38e-23  # J/K
    T=293.15       # K
    
    alpha = mu*E/(k*T)
    theta = np.linspace(0,np.pi,100) # zero: aligned with field
    ok.append(np.sum(P_theta(theta, alpha)*np.cos(theta))/np.sum(P_theta(theta, alpha)))
    



    
    

    
    
    
    
    
fig, axes = plt.subplots(2, 1, figsize=(8.5,11.5), dpi=dpi)

sns.lineplot(x=np.linspace(0.01,50,100), y=ok, ax=axes[0], color='salmon', label='Theoretical Values')
sns.lineplot(x=np.linspace(0.01,50,100), y=ok, ax=axes[1], color='salmon', label='Theoretical Values')

sns.scatterplot(x=data[:,-1], y=means, ax=axes[0], color='dodgerblue', label='Experimental Values')
sns.lineplot(x=ef, y=y, ax=axes[1], alpha=1, color='dodgerblue', label='Experimental Values')
# sns.scatterplot(x=ef, y=y, ax=axes[1], alpha=1, color='dodgerblue')


axes[0].set(xlabel=r'Electric Field ($\sigma$)', ylabel=r'Alignment of $\rho$ with EF-axis (cos($\theta$))')
axes[1].set(xlabel=r'Electric Field ($\sigma$)', ylabel=r'Alignment of $\rho$ with EF-axis (cos($\theta$))')

plt.suptitle('Alignment of H2O Dipole Moment Increases With Increasing Electric Field (30A)', fontsize=15, y=0.95)
axes[0].set_title('Box Averaged Alignments (n=3)')
axes[1].set_title('Individual Molecule Alignments w/ CI (n=887)')

axes[0].legend(loc='lower right')
axes[1].legend(loc='lower right')

plt.savefig('alignment.png')
plt.show()
plt.close(fig)

In [ ]:
mu=6.2e-30  # Cm, see http://hyperphysics.phy-astr.gsu.edu/hbase/electric/diph2o.html
E=25e8       # J/Cm (or V/m); 1e8 corresponds to 1 MV/cm
k=1.38e-23  # J/K
T=293.15       # K


alpha = mu*E/(k*T)
print(alpha)

theta = np.linspace(0,np.pi,100) # zero: aligned with field
dth = theta[1]-theta[0]

def P_theta(theta, alpha):
    return (alpha/2)*np.exp(alpha*np.cos(theta))*np.sin(theta)/np.sinh(alpha)

print(np.sum(P_theta(theta, alpha)*np.cos(theta))/np.sum(P_theta(theta, alpha)))

In [ ]:
ok = []

for E in np.linspace(0.01,50,100):
    mu=6.2e-30  # Cm, see http://hyperphysics.phy-astr.gsu.edu/hbase/electric/diph2o.html
    E*=1e8       # J/Cm (or V/m); 1e8 corresponds to 1 MV/cm
    k=1.38e-23  # J/K
    T=293.15       # K
    
    alpha = mu*E/(k*T)
    theta = np.linspace(0,np.pi,100) # zero: aligned with field
    ok.append(np.sum(P_theta(theta, alpha)*np.cos(theta))/np.sum(P_theta(theta, alpha)))
    
fig, axes = plt.subplots(1,1)
sns.lineplot(x=np.linspace(0.01,50,100), y=ok, ax=axes)
plt.show()

# 5E21

In [ ]:
# 5E21 CONDUCTIVITY

fig, axes = plt.subplots(2,1, figsize=(8.5,11.5))

df = pd.read_csv('../5E21/run2/y_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]

sns.lineplot(data=df, x='c', y='sigma', ax=axes[0], color='dodgerblue', label='Hypothetical Electric-Field Direction')
sns.scatterplot(data=df, x='c', y='sigma', ax=axes[1], color='dodgerblue', label='Hypothetical Electric-Field Direction')

df = pd.read_csv('../5E21/run1/y_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]
df['sigma'] = [-i*1000 for i in df['sigma'].tolist()]

# sns.scatterplot(data=df, x='c', y='sigma', ax=axes[0], color='salmon', label='Experimental Electric-Field Direction')



sns.lineplot(x=x, y=y, ax=axes[0], color='salmon', label='Experimental Electric-Field Direction')
sns.scatterplot(x=x, y=y, ax=axes[1], color='salmon', label='Experimental Electric-Field Direction')

axes[0].set(xlabel=r'Electric Field ($\sigma$)', ylabel=r'Conductivity (mS/cm)')
axes[1].set(xlabel=r'Electric Field ($\sigma$)', ylabel=r'Conductivity (mS/cm)')

plt.suptitle('Conductivity of PDZ2 Crystal', fontsize=15, y=0.92)

axes[0].legend(loc='upper left')
axes[1].legend(loc='upper left')

plt.savefig('temp.png')
plt.show()
plt.close(fig)

# Slanted Field

In [ ]:
converter_slant(3)

In [ ]:
def converter_slant(abt):
    slant = [0.88773502, 0., 0.46035479]
    n = slant[0] / slant[2]
    A = np.sqrt(n**2 + 1)
    
    x = abt / A
    z = n * x
    
    return x, 0, z